In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import time
import numpy as np


class AxUpdater(object):
    
    def __init__(self, fig, ax, xlim=None, ylim=None):
        
        self.fig = fig
        self.ax = ax
        
        self.xlim = xlim
        if xlim is not None:
            self.ax.set_xlim(xlim[0],xlim[1])
        
        self.ylim = ylim
        if ylim is not None:
            self.ax.set_ylim(ylim[0],ylim[1])
        
        self.trajs = []
        self.background = self.fig.canvas.copy_from_bbox(self.ax.bbox)

    def new_plot(self):
        self.trajs.append(self.ax.plot([],[], '-o')[0])
        
        
    def update_plot(self,x=None,y=None):
        assert(len(self.trajs))
        if x is not None and y is not None:
            self.trajs[-1].set_data(x, y)
        
        self.fig.canvas.restore_region(self.background)
        # redraw just the points
        self.ax.draw_artist(self.trajs[-1])
        # fill in the axes rectangle
        self.fig.canvas.blit(self.ax.bbox)
        
    def clean(self):
        for idx in range(len(self.ax.lines)-1,-1,-1):
            if self.ax.lines[idx] not in self.trajs:
                self.ax.lines.pop(idx)
        for idx in range(len(self.ax.collections)-1,-1,-1):
            self.ax.collections.pop(idx)
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
        
    def clear(self):
        self.ax.set_prop_cycle(None)
        self.trajs=[]
        self.clean()

class DrawOnAx(AxUpdater):

    def __init__(self, fig, ax, xlim=(0.0,1.0),ylim=(0.0,1.0)):
        AxUpdater.__init__(self, fig, ax, xlim, ylim)
        
        self.curr_x = None
        self.curr_y = None
        self.curr_t = None
        self.curr_t_start = None
        self.data = []
        self.cbs = []

        self.do_record = False
        
        self.fig.canvas.mpl_connect("button_press_event", self.on_press)
        self.fig.canvas.mpl_connect("button_release_event", self.on_release)
        self.fig.canvas.mpl_connect("motion_notify_event", self.on_move)

    def add_cb(self,cb):
        assert(isinstance(cb,tuple) and len(cb)==3)
        
        self.cbs.append(cb)
        
    def on_press(self, event):
        if event.inaxes != self.ax:
            return
        if self.do_record:
            return
        self.curr_t_start = time.time() 
        self.curr_t = [0.0]
        self.curr_x = [event.xdata]
        self.curr_y = [event.ydata]
        self.new_plot()
        self.update_plot(self.curr_x,self.curr_y)
        self.do_record=True
        
        for cb in self.cbs:
            if cb[0] is not None:
                cb[0](self.curr_t,self.curr_x,self.curr_y)

    def on_move(self, event):
        if event.inaxes != self.ax:
            return
        if not self.do_record:
            return
        
        self.curr_t.append(time.time()-self.curr_t_start)
        self.curr_x.append(event.xdata)
        self.curr_y.append(event.ydata)
        self.update_plot(self.curr_x,self.curr_y)
        
        for cb in self.cbs:
            if cb[1] is not None:
                cb[1](self.curr_t,self.curr_x,self.curr_y)

    def on_release(self, event):
        if event.inaxes != self.ax:
            return
        if not self.do_record:
            return
        self.do_record = False
        self.curr_t.append(time.time()-self.curr_t_start)
        self.curr_x.append(event.xdata)
        self.curr_y.append(event.ydata)
        self.update_plot(self.curr_x,self.curr_y)
        self.data.append(np.array([self.curr_t, self.curr_x, self.curr_y]))
        
        for cb in self.cbs:
            if cb[2] is not None:
                cb[2](self.curr_t,self.curr_x,self.curr_y)


    def clear(self):
        self.data = []
        AxUpdater.clear(self)



# Motivation
$\newcommand\traj{\boldsymbol{\tau}}$
Given:
- Trajectory $\traj \in \mathbb{R}^{D\times L}$, with dimension $D$, over $L$ time steps

Desired: Movement representation that
- is parametrized
- is time invariant
- considers multiple demonstrations
- captures correlations between DoFs





In [7]:
fig, axs = plt.subplots(3,1,squeeze=False)

doa = DrawOnAx(fig, axs[0][0])
uax = AxUpdater(fig, axs[1][0],xlim=(0.0,3.0),ylim=(0.0,1.0))
uay = AxUpdater(fig, axs[2][0],xlim=(0.0,3.0),ylim=(0.0,1.0))

doa.add_cb((lambda t,x,y: (uax.new_plot(),uax.update_plot(t,x)), lambda t,x,y: uax.update_plot(t,x), lambda t,x,y: uax.update_plot(t,x)))
doa.add_cb((lambda t,x,y: (uay.new_plot(),uay.update_plot(t,y)), lambda t,x,y: uay.update_plot(t,y), lambda t,x,y: uay.update_plot(t,y)))


## Parametrized
$\newcommand\basis{\boldsymbol{\phi}}$
$\newcommand\w{\boldsymbol{w}}$
$\newcommand\phase{\boldsymbol{z}}$
- Considering each time step individually is often infeasible
    - large parameter space
    - how to ensure smoothness?
    - how to achieve time invariance?
    
- Parameterizing the trajectory using a fixed number $B$ of basis functions
    - $\traj = \basis(\phase)\w$
        - $\phase$: phase, i.e. (time)steps at which to compute $\tau$
        - $\basis(\phase)$: basis, e.g., normalized radial: $\phi_i(z_t) = \dfrac{b_i(z_t)}{\sum_j b_j(z_t)}$, $b_i=\exp\left(-\dfrac{(z_t-c_i)^2}{2h}\right)$
        - $\w$: weights


In [8]:
# Radial basis function
def radial_basis(phase,centers,bandwidth):
    bases = np.exp(-(np.repeat(phase[...,np.newaxis],centers.shape,-1)-centers)**2/(2*bandwidth)).T
    bases /= bases.sum(axis=0)
    return bases.T
    
# The recorded data
data = doa.data[-1]

# interpolated data...
# usually not required since recorded data
# comes in certain frequency!!    
hz = 100
def interp_data(data,hz):
    t = np.arange(data[0,0],data[0,-1],1/hz)
    x = np.interp(t,data[0,:],data[1,:])
    y = np.interp(t,data[0,:],data[2,:])
    print("data is: " , data)
    print("t is: ", t)
    print("x is: " , x)
    print("y is: " , y)
    return np.array([t,x,y]) 



# Number of bases
B = 15

# The timesteps for the basis functions (phase)
z = np.linspace(data[0,0],data[0,-1],100)

# Equidistant centers for the bases
c = np.linspace(z[0],z[-1],B)

# A heursitic for a possible/plausible bandwidth (bandwidth)
h = -(c[1]-c[0])**2/(2*np.log(0.3))



#basis functions
ϕ = radial_basis(z,c,h)
print("ϕ.shape: {}".format(ϕ.shape))
w = np.ones(B)


doa.clean()
doa.ax.plot(data[1,:],data[2,:],'b-o',linewidth=3)

uax.clean()
uax.ax.plot(data[0,:],data[1,:],'b-o',linewidth=3)
uax.ax.plot(z,ϕ*w,linewidth=3)
uax.ax.plot(z,np.dot(ϕ,w),'g',linewidth=3)


uay.clean()
uay.ax.plot(data[0,:],data[2,:],'b-o',linewidth=3)
uay.ax.plot(z,ϕ*w,linewidth=3)
uay.ax.plot(z,np.dot(ϕ,w),'g',linewidth=3)


ϕ.shape: (100, 15)


<span style="color:#AA0000; font-weight:bold;">Obviously, something is wrong!</span>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## Learning the weights $\w$

- straight forward approach learn the weights via ridge regression
    -- $\w = \left(\basis(\phase)^T\basis(\phase) + \lambda I\right)^{-1} \basis(\phase)^T\traj$


In [9]:

z = data[0,:]
ϕ = radial_basis(z,c,h)
    

# ridge regression
def learn_weights_1(data, ϕ, λ=1e-6):
    wx = np.dot(np.linalg.inv(np.dot(ϕ.T,ϕ)+λ*np.eye(B)),np.dot(ϕ.T,data[1,:]))
    wy = np.dot(np.linalg.inv(np.dot(ϕ.T,ϕ)+λ*np.eye(B)),np.dot(ϕ.T,data[2,:]))
    return np.array([wx,wy])

w1 = learn_weights_1(data, ϕ)
# also ridge regression but numerically more stable
def learn_weights_2(data, ϕ, λ=1e-6):
    wx = np.linalg.solve(np.dot(ϕ.T,ϕ)+λ*np.eye(B),np.dot(ϕ.T,data[1,:]))
    wy = np.linalg.solve(np.dot(ϕ.T,ϕ)+λ*np.eye(B),np.dot(ϕ.T,data[2,:]))
    return np.array([wx,wy])

w2 = learn_weights_2(data, ϕ)

print("max abs difference between inv and solve: {}".format(np.max(np.abs(w1-w2))))

# still ridge regression but for all dimensions at once
def learn_weights(data, ϕ, λ=1e-6):
    w = np.linalg.solve(np.dot(ϕ.T,ϕ)+λ*np.eye(B),np.dot(ϕ.T,data[1:,:].T)).T
    return w

w = learn_weights(data, ϕ)

print("w.shape: {}".format(w.shape))
print("max abs difference between solve and solve for all dims: {}".format(np.max(np.abs(w-w2))))

uax.clean()
uax.ax.plot(data[0,:],data[1,:],'r-x',linewidth=3)
uax.ax.plot(z,ϕ*w[0,:],linewidth=3)
uax.ax.plot(z,np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(data[0,:],data[2,:],'r-x',linewidth=3)
uay.ax.plot(z,ϕ*w[1,:],linewidth=3)
uay.ax.plot(z,np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(data[1,:],data[2,:],'r-x',linewidth=3)
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

max abs difference between inv and solve: 2.220446049250313e-15
w.shape: (2, 15)
max abs difference between solve and solve for all dims: 3.2751579226442118e-15


<span style='color:green; font-weight:bold;'>Because of the parametrization we can reproduce the trajectory in a different resolution</span>

In [10]:
z = np.arange(data[0,0],data[0,-1],1/10)

ϕ = radial_basis(z,c,h)

uax.clean()
uax.ax.plot(z,ϕ*w[0,:],linewidth=3)
uax.ax.plot(z,np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(z,ϕ*w[1,:],linewidth=3)
uay.ax.plot(z,np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

## Time Invariance

- How can we execute a learned trajectory faster or slower?
    1. extend phase $z$
    2. recompute centers $c$
    3. set new bandwidth $h$
    4. recompute basis

In [11]:
des_duration = 2

z = np.arange(data[0,0],des_duration,1/hz)
c = np.linspace(z[0],z[-1],B)
h = -(c[1]-c[0])**2/(2*np.log(0.3))

ϕ = radial_basis(z,c,h)

uax.clean()
uax.ax.plot(z,ϕ*w[0,:],linewidth=3)
uax.ax.plot(z,np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(z,ϕ*w[1,:],linewidth=3)
uay.ax.plot(z,np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

### Simpler alternative:
- temporal scaling of each trajectory, such that it was always executed between $0.0$ and $1.0$
    - centers never have to change
    - bandwidth never has to change
    - phase is just normalized time



In [12]:
def get_phase(t):
    phase = t-np.min(t)
    phase /= np.max(phase)
    return phase

# Equidistant centers for the bases
c = np.linspace(0.0,1.0,B)
# A heursitic for a possible/plausible bandwidth
h = -(1/(B-1))**2/(2*np.log(0.3))

z = get_phase(data[0,:])
ϕ = radial_basis(z,c,h)

w = learn_weights(data,ϕ)

uax.clean()
uax.ax.plot(data[0,:],data[1,:],'r-x',linewidth=3)
uax.ax.plot(data[0,:],ϕ*w[0,:],linewidth=3)
uax.ax.plot(data[0,:],np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(data[0,:],data[2,:],'r-x',linewidth=3)
uay.ax.plot(data[0,:],ϕ*w[1,:],linewidth=3)
uay.ax.plot(data[0,:],np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(data[1,:],data[2,:],'r-x',linewidth=3)
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

reproductions in different velocities are now simpler
    - compute phase of desired time axis
    - recompute basis

In [13]:

des_t = np.arange(0.0,des_duration,1/hz)

z = get_phase(des_t)
ϕ = radial_basis(z,c,h)


uax.clean()
uax.ax.plot(data[0,:],data[1,:],'r-x',linewidth=3)
uax.ax.plot(des_t,ϕ*w[0,:],linewidth=3)
uax.ax.plot(des_t,np.dot(ϕ,w[0,:]),'g-s',linewidth=3)


uay.clean()
uay.ax.plot(data[0,:],data[2,:],'r-x',linewidth=3)
uay.ax.plot(des_t,ϕ*w[1,:],linewidth=3)
uay.ax.plot(des_t,np.dot(ϕ,w[1,:]),'g-s',linewidth=3)


doa.clean()
doa.ax.plot(data[1,:],data[2,:],'r-x',linewidth=3)
doa.ax.plot(np.dot(ϕ,w[0,:]),np.dot(ϕ,w[1,:]),'g-s',linewidth=3)

interp_data(data, hz)

data is:  [[0.         0.11923695 0.15674901 0.1706059  0.18860483 0.20307994
  0.22240496 0.23668885 0.25421381 0.28864312 0.30327082 0.32143497
  0.33474493 0.35460901 0.36769199 0.38849306 0.42421198 0.45699787
  0.471488   0.50560594 0.53942394 0.55436397 0.58940601 0.62343192
  0.65737391 0.68820882 0.70303202 0.72111487 0.73460889 0.77228403
  0.80631399 0.82048321 0.83773017 0.85151792 0.87199283 0.90650487
  0.93850303 0.95313096 0.97200203 0.98679781 1.00468206 1.01910782
  1.03870177 1.05337501 1.07220411 1.08668399 1.10593104 1.12079215
  1.13806605 1.15337181 1.1706109  1.18471718 1.20386696 1.21754599
  1.23875904 1.25357604 1.27186489 1.28652191 1.30536985 1.31981897
  1.33834291 1.35096717 1.37007117 1.40569091 1.43841481 1.453655
  1.47152805 1.50756884 1.53982496 1.55440378 1.56720495 1.58894801]
 [0.0483871  0.0483871  0.05040323 0.05443548 0.05846774 0.06451613
  0.06854839 0.0766129  0.08467742 0.10685484 0.11895161 0.12701613
  0.13709677 0.14717742 0.15725806 0.16

array([[0.        , 0.01      , 0.02      , 0.03      , 0.04      ,
        0.05      , 0.06      , 0.07      , 0.08      , 0.09      ,
        0.1       , 0.11      , 0.12      , 0.13      , 0.14      ,
        0.15      , 0.16      , 0.17      , 0.18      , 0.19      ,
        0.2       , 0.21      , 0.22      , 0.23      , 0.24      ,
        0.25      , 0.26      , 0.27      , 0.28      , 0.29      ,
        0.3       , 0.31      , 0.32      , 0.33      , 0.34      ,
        0.35      , 0.36      , 0.37      , 0.38      , 0.39      ,
        0.4       , 0.41      , 0.42      , 0.43      , 0.44      ,
        0.45      , 0.46      , 0.47      , 0.48      , 0.49      ,
        0.5       , 0.51      , 0.52      , 0.53      , 0.54      ,
        0.55      , 0.56      , 0.57      , 0.58      , 0.59      ,
        0.6       , 0.61      , 0.62      , 0.63      , 0.64      ,
        0.65      , 0.66      , 0.67      , 0.68      , 0.69      ,
        0.7       , 0.71      , 0.72      , 0.73

Defining a normalized phase also makes it easier to combine multiple demonstrations


## Considering Multiple Demonstrations

- We consider each demonstration as a 'variation' or 'sample' of the same movement primitive
- hence, we treat the weight vector for each demonstration $\w_i$ as an instance of a random variable, drawn from a multivariate Gaussian $\w_i \sim \mathcal{N}(\w|\mu_w,\Sigma_w)$

In [25]:
doa.clear()
uax.clear()
uay.clear()



In [36]:

# Equidistant centers for the bases
c = np.linspace(0.0,1.0,B)
# A heursitic for a possible/plausible bandwidth
h = -(1/(B-1))**2/(2*np.log(0.3))

trajectories = [interp_data(d,hz) for d in doa.data]

def learn_weight_distribution(trajectories):
    ws = np.array([learn_weights(d,radial_basis(get_phase(d[0,:]),c,h)).flatten() for d in trajectories])
    μ = np.mean(ws,axis=0)
    Σ = np.cov(ws.T)
    return μ, Σ

μ_w, Σ_w = learn_weight_distribution(trajectories)




data is:  [[0.         0.11861396 0.14658785 0.16041684 0.1792779  0.19404578
  0.21297097 0.22731495 0.24630976 0.26075101 0.27847075 0.29418492
  0.33048606 0.34480882 0.36317587 0.37749505 0.39539099 0.41090989
  0.43060088 0.46406078 0.47844291 0.49680591 0.51086998 0.52978396
  0.56115603 0.59646583 0.61065292 0.63004375 0.64451981 0.6638
  0.67889571 0.7126689  0.72726798 0.76325393 0.77804279 0.79641604
  0.81110907 0.82935905 0.84389496 0.86288881 0.87745881 0.89647412
  0.91156888 0.947083   0.97841406 0.99359488 1.03004885 1.06208396
  1.07551885 1.09493709 1.11332989 1.12747598 1.16389179 1.19657588
  1.21131182 1.22982502 1.24430895 1.26302314 1.27736282 1.29713392
  1.31080699 1.32957315 1.36286211 1.3966279  1.41142297 1.44741201
  1.48154998 1.49660897 1.53118801 1.56475997 1.57783484]
 [0.06451613 0.06451613 0.06451613 0.06451613 0.06451613 0.06653226
  0.06854839 0.07459677 0.08064516 0.08266129 0.09072581 0.09677419
  0.1108871  0.11693548 0.12298387 0.12701613 0.1310

We can now sample from the weight distribution and produce new similar trajectories

In [45]:
doa.clean()
uax.clean()
uay.clean()

des_duration = 2
des_t = np.arange(0.0,des_duration,1/hz)

z = get_phase(des_t)
ϕ = radial_basis(z,c,h)

ws = np.random.multivariate_normal(μ_w, Σ_w, 10)
ws = ws.reshape(ws.shape[0],-1,B)
print(ws.shape)
uax.ax.plot(des_t,np.dot(ϕ,ws[:,0,:].T),linewidth=3)
uay.ax.plot(des_t,np.dot(ϕ,ws[:,1,:].T),linewidth=3)
doa.ax.plot(np.dot(ϕ,ws[:,0,:].T),np.dot(ϕ,ws[:,1,:].T),linewidth=3)

(10, 2, 15)


We can even write down the distribution in the original trajectory space:
- $\traj \sim \mathcal{N}(\traj|\mu_\tau,\Sigma_\tau)$
    - $\mu_\tau = \Psi\mu_w$
    - $\Sigma_\tau = \Psi\Sigma_w\Psi^T+\Sigma_{\mathrm{obs}}$
    - $\Psi$: block diagonal of $D$ blocks. Each block being $\phi$

In [47]:


def get_traj_distribution(μ_w, Σ_w, Ψ, des_duration=1.0):
    μ = np.dot(Ψ,μ_w)
    Σ = np.dot(np.dot(Ψ,Σ_w),Ψ.T)
    return μ, Σ

des_t = np.arange(0.0,des_duration,1/hz)
z = get_phase(des_t)
ϕ = radial_basis(z,c,h)
D = 2
Ψ = np.kron(np.eye(int(μ_w.shape[0]/B),dtype=int),ϕ)

μ_τ, Σ_τ = get_traj_distribution(μ_w, Σ_w, Ψ, des_duration)

print(μ_τ.shape)
des_t = np.arange(0.0,des_duration,1/hz)
μ_D = μ_τ.reshape((-1,des_t.shape[0]))
print(μ_D.shape)

uax.ax.plot(des_t,μ_D[0,:],'m',linewidth=5)
uay.ax.plot(des_t,μ_D[1,:],'m',linewidth=5)

print(np.diag(Σ_τ).shape)
σ_τ = np.sqrt(np.diag(Σ_τ))
σ_D = σ_τ.reshape((-1,des_t.shape[0]))

uax.ax.fill_between(des_t, μ_D[0,:]-2*σ_D[0,:], μ_D[0,:]+2*σ_D[0,:], color='m',alpha=0.3)
uay.ax.fill_between(des_t, μ_D[1,:]-2*σ_D[1,:], μ_D[1,:]+2*σ_D[1,:], color='m',alpha=0.3)


(400,)
(2, 200)
(400,)


In [48]:
doa.clean()
uax.clean()
uay.clean()

#implemented function, might still have to mess with matrices dimensions to work
def conditioning(μ_w, Σ_w, y_t, Σ_y, Ψ):
    
    L = (Σ_w.dot(Ψ)).dot(np.linalg.inv(Σ_y + (Ψ.T).dot(Σ_w).dot(Ψ)))
    
    new_μ_w = μ_w + L.dot(y_t - (Ψ.T).dot(μ_w))
    new_Σ_w = Σ_w - L.dot(Ψ.T).dot(Σ_w)
    return new_μ_w, new_Σ_w

Ψ = np.kron(np.eye(int(μ_w.shape[0]/B),dtype=int),ϕ)

#call to function
μ_τ, Σ_τ = conditioning(μ_τ, Σ_τ, 0.5, .1, Ψ)


#Ψ = np.kron(np.eye(int(μ_τ.shape[0]/B),dtype=int),ϕ)

#get new mean with conditioning
#μ_τ = np.dot(Ψ.T,μ_τ)
#get new distribution with conditioning
#Σ_τ = np.dot(np.dot(Ψ,Σ_τ),Ψ.T)

print(μ_τ.shape)
des_t = np.arange(0.0,des_duration,1/hz)
μ_D = μ_τ.reshape((-1,des_t.shape[0]))
print(μ_D.shape)

uax.ax.plot(des_t,μ_D[0,:],'m',linewidth=5)
uay.ax.plot(des_t,μ_D[1,:],'m',linewidth=5)

#print(np.diag(Σ_τ).shape)
#σ_τ = np.sqrt(np.diag(Σ_τ))
#σ_D = σ_τ.reshape((-1,des_t.shape[0]))

#uax.ax.fill_between(des_t, μ_D[0,:]-2*σ_D[0,:], μ_D[0,:]+2*σ_D[0,:], color='m',alpha=0.3)
#uay.ax.fill_between(des_t, μ_D[1,:]-2*σ_D[1,:], μ_D[1,:]+2*σ_D[1,:], color='m',alpha=0.3)

(400,)
(2, 200)
